In [1]:
import sys
import csv
sys.path.append('..')
from BDDData import *
import numpy as np
import torch
import torch.utils.data as data

import networkx as nx
import matplotlib.pyplot as plt
from scipy.linalg import circulant
from product_graph import *
from utils import *

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
#Load dataframes
bdd_data = BDD_dataset("raw_data/BDDdata/")
#Add column with the timestep
bdd_data.add_timestep_id()
#Add flags for chaotic values
bdd_data.tag_chaotic(replace=True)
#Compute the mod for the nazelle and wind angles
bdd_data.angle_mod()
#Interpolate the missing values
bdd_data.interpolate_power()
#Values smaller than 0 are set to 0
bdd_data.cap_power_to_zero()
#Normalize Patv feature to [0,1]
bdd_data.normalize_power(min=0, max=1, method= "MinMaxScaler")
#Convert df to matrix form, where only Patv is included. Then split into train, validation and test
#The matrix contains the subset of the time series for ALL nodes, so an (TxN matrix)
train, val, test = bdd_data.split_df()

In [4]:
# set seed
np.random.seed(42)
# consider a p percentage of the data
p = 1.0
train_mask = np.random.choice(train.shape[1], int(train.shape[1] * p), replace=False)
val_mask = np.random.choice(val.shape[1], int(val.shape[1] * p), replace=False)
test_mask = np.random.choice(test.shape[1], int(test.shape[1] * p), replace=False)

train = train[:, train_mask]
val = val[:, val_mask]
test = test[:, test_mask]

In [5]:
class CustomBDD_Dataset(data.Dataset):
    def __init__(self, dataset, observation_window=12, forecast_window=12, starting_turbine = 0,  ending_turbine=133):
        self.observation_window = observation_window
        self.forecast_window = forecast_window
        length = eval(f'len({dataset}[0])')#Retrieves length of dataset
        bdd_data.get_observation_forecasting_window(time_series_len=length, observation_steps=self.observation_window, forecast_steps=self.forecast_window)#Generates obs window
        self.window_of_interest =  bdd_data.sliding_indices[str(self.observation_window)+","+str(self.forecast_window)]#Retrieves windows
        self.starting_turbine = starting_turbine
        self.ending_turbine = ending_turbine  
        self.dataset = dataset

    def __len__(self):
        return len(self.window_of_interest)

    def __getitem__(self, idx):
        window = self.window_of_interest[idx]
        if self.dataset == "train":
            features = train[self.starting_turbine:self.ending_turbine+1,window[0]:window[1]]
            labels = train[self.starting_turbine:self.ending_turbine+1,window[1]:window[2]]
        elif self.dataset == "val":
            features = val[self.starting_turbine:self.ending_turbine+1,window[0]:window[1]]
            labels = val[self.starting_turbine:self.ending_turbine+1,window[1]:window[2]]
        elif self.dataset == "test":
            features = test[self.starting_turbine:self.ending_turbine+1,window[0]:window[1]]
            labels = test[self.starting_turbine:self.ending_turbine+1,window[1]:window[2]]
        else:
            raise NotImplementedError
        return torch.from_numpy(features).float(), torch.from_numpy(labels).float()
    
obs_window = 12
forecast_window = 12
batch_size = 100
num_nodes = 134

train_dataset = CustomBDD_Dataset("train",observation_window=obs_window,forecast_window=forecast_window)
train_loader = data.DataLoader(train_dataset, shuffle=True, batch_size = batch_size)
val_dataset = CustomBDD_Dataset("val",observation_window=obs_window,forecast_window=forecast_window)
val_loader = data.DataLoader(val_dataset, shuffle=True, batch_size = batch_size)

In [6]:
x,y = next(iter(train_loader))
print(f"{x.shape=}\n{y.shape=}")

x.shape=torch.Size([100, 134, 12])
y.shape=torch.Size([100, 134, 12])


In [7]:
G = nx.read_gml('data/spatial_graph_2000.gml')
adj_mat = nx.adjacency_matrix(G)
adj_mat = nx.to_numpy_array(G)

In [8]:
S = normalize_adjacency(torch.tensor(adj_mat)).float().to(device)

In [9]:
print(adj_mat.shape)

(134, 134)


In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GCNNLayer(nn.Module):
    def __init__(self, in_features, out_features, matrix_powers, order):
        super(GCNNLayer, self).__init__()
        self.matrix_powers = matrix_powers
        self.order = order
        self.weights = nn.Parameter(torch.FloatTensor(in_features, out_features, order))
        # use Xavier initialization to match variance of input with output
        nn.init.xavier_uniform_(self.weights)

    def forward(self, features):
        batch_size = features.size(0)
        output_dim = self.weights.size(1)
        device = features.device

        out = torch.zeros((batch_size, features.size(1), output_dim), device=device)
        for k in range(self.order):
            weighted = torch.bmm(features, self.weights[:, :, k].unsqueeze(0).repeat(batch_size, 1, 1))
            shifted = torch.bmm(self.matrix_powers[k].unsqueeze(0).repeat(batch_size, 1, 1).to(device), weighted)
            out += shifted
        return out

# Inputs must be sized [num_nodes, obs_size] and outputs will be [num_nodes, pred_size]
class GCNN(nn.Module):
    def __init__(self, hid_sizes, shift, order):
        super(GCNN, self).__init__()
        self.layers = nn.ModuleList()
        # compute matrix shift
        matrix_powers = [torch.matrix_power(shift, k).float() for k in range(order)]
        # input layer of size obs_size
        self.layers.append(GCNNLayer(obs_window, hid_sizes[0], matrix_powers, order))
        # num_hid hidden layers of size hid_size
        for i in range(len(hid_sizes) - 1):
            self.layers.append(GCNNLayer(hid_sizes[i], hid_sizes[i + 1], matrix_powers, order))

    def forward(self, features):
        temp = features
        for layer in self.layers[:-1]:
            # use relu activation function
            temp = F.relu(layer(temp))
        return self.layers[-1](temp)

model = GCNN([128,128, forecast_window], S,1).to(device)

pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Created GCNN model with {pytorch_total_params} parameters:")

for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data.size())

Created GCNN model with 97280 parameters:
layers.0.weights torch.Size([12, 128, 5])
layers.1.weights torch.Size([128, 128, 5])
layers.2.weights torch.Size([128, 12, 5])


In [20]:
import time

def train_epoch(model, loader, optimizer, device='cpu'):
    model.train()
    total_loss = 0

    for x, y in loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        outputs = model(x)
        loss = torch.nn.functional.mse_loss(outputs, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def evaluate_epoch(model, loader, device='cpu'):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            loss = torch.nn.functional.mse_loss(outputs, y)
            total_loss += loss.item()
    return total_loss / len(loader)

def train_model(model, train_loader, val_loader, num_epochs=10):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=5e-4)

    start_time = time.time()
    train_losses = []
    val_losses = []

    for epoch in range(1, num_epochs + 1):
        train_loss = train_epoch(model, train_loader, optimizer, device=device)
        val_loss = evaluate_epoch(model, val_loader, device=device)
        train_losses.append(train_loss)
        val_losses.append(val_loss)

        if epoch % 1 == 0:
            print(f"epoch: {epoch}\ttraining loss: {train_loss:.4f}\tvalidation loss: {val_loss:.4f}")

    elapsed_time = time.time() - start_time
    if device.type == 'cuda':
        torch.cuda.synchronize()
    print(f'Model training took {elapsed_time:.3f} seconds')

    return train_losses, val_losses

train_losses, val_losses = train_model(model, train_loader, val_loader)

epoch: 1	training loss: 3074.3717	validation loss: 0.0975
epoch: 2	training loss: 0.1309	validation loss: 0.0975
epoch: 3	training loss: 0.9947	validation loss: 0.0974
epoch: 4	training loss: 0.1309	validation loss: 0.0974
epoch: 5	training loss: 0.1317	validation loss: 0.0974
epoch: 6	training loss: 0.1309	validation loss: 0.0975
epoch: 7	training loss: 0.1309	validation loss: 0.0974
epoch: 8	training loss: 0.1309	validation loss: 0.0975
epoch: 9	training loss: 1.6101	validation loss: 0.0975
epoch: 10	training loss: 0.1309	validation loss: 0.0975
Model training took 32.583 seconds


In [21]:
# Save the model
torch.save(model.state_dict(), 'results/gcnn_e=10_d=1.0_k=1_[128,128,12].pth')

In [ ]:
# load the model
# model = model.load_state_dict(torch.load('model_name.pth'))